# Portfolio
Brian Howard
26 March 2022

In [1]:
library(UsingR)
library(quantmod)
library(ggplot2)
library(grid)
library(gridExtra)
library(knitr)
library(kableExtra)
library(tools)
library(zoo)
library(signal)
library(stringr)
library(corrplot)
library(Hmisc)
library(tseries)
library(caret)
library(rpart)
library(rpart.plot)
library(randomForest)
library(Quandl)
library(nnet)
#library(devtools)
#install_github("andrewuhl/RollingWindow")
library(RollingWindow)
library(gtable)
library(data.table)
library(readxl)
library(IRdisplay)

Loading required package: MASS

Loading required package: HistData

Loading required package: Hmisc

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: 'Hmisc'


The following objects are masked from 'package:base':

    format.pval, units



Attaching package: 'UsingR'


The following object is masked from 'package:survival':

    cancer


Loading required package: xts

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: TTR

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: 'quantmod'


The following object is masked from 'package:Hmisc':

    Lag



Attaching package: 'signal'


The following objects are masked from 'package:stats':

    filter, poly


corrplot 0.92 loaded

Warning message:

Define the source of the data (local or web) and whether to update the models.

Call helper functions

In [2]:
source("plotHelper.r")

These functions help organize tables in the document

In [3]:
tblPort <- function(rrData, datay){
  
  df.port <- rrData[[1]][c("string.symbol", "string.description", "date.series.start", "float.expense.ratio",
                      datay, paste(datay, "_Opt", sep = ""))]
  
  string.colnames <- colnames(df.port);
  string.colnames[1] <- "Symbol"
  string.colnames[2] <- "Description"
  string.colnames[3] <- "Series Start"
  string.colnames[4] <- "Expense Ratio (%)"

  kable(rrData[[1]][c("string.symbol", "string.description", "date.series.start", "float.expense.ratio",
                      datay, paste(datay, "_Opt", sep = ""))], digits=c(0,0,3,3,3,3)) %>%
  kable_styling(bootstrap_options = c("striped", "hover"))
  
}


Define the source of the data (local or web) and whether to update the models.

In [4]:
b.refresh.models = FALSE

Define the plotting ranges and vertical defaults

In [5]:
dt.recent = as.Date("2015-01-01")
string.analysis.start <- "2007-05-22"
iRetPd <- 180
d.GSPC.max = 3500

## Load the data and create the data frame
Load up the off-line data

In [6]:
load("C:/Users/Rainy/OneDrive/RecessionIndicator_Buffer.RData")

This snippet is needed because some of the ticker symbols include are invalid variable names so re-cast those variable names.

In [7]:
df.symbols$string.symbol <-
  str_replace_all(df.symbols$string.symbol, "[^[:alnum:]]", "")

Do the interpolation to daily samples and tidy the data

In [8]:
source("calcInterpolate.r")

In [9]:
df.data <- calcInterpolate(df.symbols)

Warning message in merge.xts(xtsData, get(df.symbols$string.symbol[idx])):
"NAs introduced by coercion"


Truncate the data. A very few data series do go back to 1854, but most don't even go past WWII so truncate the dataframe

In [10]:
df.data <- df.data[as.Date(rownames(df.data)) > as.Date("1940-01-01"),]

Create aggreagate data series
Some analysis requires that two or more series be combined. For example, normallizing debt by GDP to get a sense of the proportion of debt to the total economy helps understand the debt cycle.

## Normalize data
Linearly interpolate the data, add normalized columns (this helps speed portfolio calcs)

In [11]:
for (col_name in names(df.data))
{
    if (is.numeric(df.data[,col_name]))
    {
      # Normalize data
      str.col.norm <- paste(col_name, "_Norm", sep="") 
      d.den = df.data[as.Date(rownames(df.data)) == as.Date(string.analysis.start),col_name]
        if (abs(d.den) > 0.00000001){
            df.data[, str.col.norm] <- df.data[,col_name] / d.den
        }else{
            df.data[, str.col.norm] <- 1
        }
      
        
     # Descriptions must be associated with the root data series name. If the
     # symbol table is empty and there is a "." in the name, look up the root
     # symbol and use that.
     str.symbol.root <- col_name
       if (grepl("\\.", col_name)) {
         str.symbol.root <- substr(col_name, 1, regexpr("\\.", col_name) - 1)
       }
        
      # Add it to the symbols table
      str.desc <- paste(df.symbols$string.description[df.symbols$string.symbol == str.symbol.root], "\nNormalized", sep =
                                       "")
      df.symbols <-
      rbind(
        df.symbols,
        data.frame(
          string.symbol = str.col.norm,
          string.source = "Calc",
          string.description = str.desc,
          string.label.y = "-",
          float.expense.ratio = -1.00,
          date.series.start = df.symbols$date.series.start[df.symbols$string.symbol == str.symbol.root],
          date.series.end = df.symbols$date.series.end[df.symbols$string.symbol == str.symbol.root]
        )
      )
    
    }
}

In [12]:
source("calcAggregateSeries.r")

Calculate the year-over-year, 50 day moving average, and other features in the data

In [13]:
source("calcFeatures.r")

In [ ]:
lst.df <- calcFeatures(df.data, df.symbols)
df.data <- lst.df[[1]]
df.symbols <- lst.df[[2]]

These are auxilliary series that use the YoY and smoothed data.

In [ ]:
source("calcFeaturesAggregate.r")

Build the recession data frame

In [ ]:
source("calcRecession.r")

Portfolio specific functions

In [ ]:
source("calcPort.r")

# Introduction
This is a portfolio analysis, data pulled from yahoo. I’m trying to decide what a better mix would be for this old IRA account. These next few segments load the data into the R program.

## Define the symbols to be used in the analysis.

This code defines stock ticker symbols that will be available for analysis. It includes descriptions and labels for the plots.

The table uses the analysis start date as the series start date to instantiate the data frame. Once the data is pulled down into the zoo object the actual series start data will replace the analyis start.

## Truncate data

A few last calculations to truncate and aggregrate the data

In [ ]:
df.data <- with(df.data, df.data[date>=string.analysis.start,])

# Calculate the returns

## Calculate the time based returns on annual basis

In this analysis all the returns are calculated on an annual basis. It means there is a year of dead data, but it eliminates seasonal variation in data (think: Christmas).

## Summarize returns

In constructing the portfolio it is importance to understand the blend of funds. We want maximum returns for minimum volatility. That's not c


In [ ]:
# Add normalized columns (this helps speed portfolio calcs)
for (col_name in names(df.data))
{
    if (is.numeric(df.data[,col_name]))
    {

      # Split the name ("USGFG.Close"" is "USGFG"" and "Close"")
      lstSyms <- lstSymSplit(col_name)
        
      # Only if there is two terms
      if( length(lstSyms) > 1){

        if( lstSyms[2] == 'Close_YoY' ){

          # Return is the mean of the series
          df.symbols[df.symbols$string.symbol==lstSyms[1],'ExpReturn'] <- mean(df.data[, col_name])

          # Volatility is the standard deviation of the series.
          df.symbols[df.symbols$string.symbol==lstSyms[1],'Volatility'] <- sd(df.data[, col_name])
          
        }
      }

    }
}



# Define the various portfolios

This code is evaluating ROTH account portfolios. IVOO has a lower expense ratio than MDY, but performance is similar and MDY has a history that goes back to 1995 so I am using MDY as a substitute fo IVOO.

In [ ]:
df.symbols$pf_Roth_Legacy <- 0
df.symbols[df.symbols$string.symbol=='VTWO',]$pf_Roth_Legacy <-  0.0000
df.symbols[df.symbols$string.symbol=='VFINX',]$pf_Roth_Legacy <-  0.5051
df.symbols[df.symbols$string.symbol=='TMFGX',]$pf_Roth_Legacy <-  0.0000
df.symbols[df.symbols$string.symbol=='IWM',]$pf_Roth_Legacy <-  0.0001
df.symbols[df.symbols$string.symbol=='QQQ',]$pf_Roth_Legacy <-  0.0001
df.symbols[df.symbols$string.symbol=='HAINX',]$pf_Roth_Legacy <-  0.0001
df.symbols[df.symbols$string.symbol=='VEU',]$pf_Roth_Legacy <-  0.0000


df.symbols$pf_Roth_IRA <- 0
df.symbols[df.symbols$string.symbol=='VTWO',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='EES',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='IJR',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='VFINX',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='VOE',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='VOT',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='NOBL',]$pf_Roth_IRA <-  0.283
df.symbols[df.symbols$string.symbol=='GDX',]$pf_Roth_IRA <-  0.0566
df.symbols[df.symbols$string.symbol=='SCHD',]$pf_Roth_IRA <-  0.283
df.symbols[df.symbols$string.symbol=='XLE',]$pf_Roth_IRA <-  0.0943
df.symbols[df.symbols$string.symbol=='PFF',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='HPI',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='VEU',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='IWM',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='QQQ',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='ONEQ',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='IVOO',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='MDY',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='VO',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='CZA',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='BIL',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='VYM',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='ACWI',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='VEU',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='DJI',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='RLG',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='SPY',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='PRWCX',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='BSV',]$pf_Roth_IRA <-  0.0000
df.symbols[df.symbols$string.symbol=='BIV',]$pf_Roth_IRA <-  0.283

# Amy's 401K
df.symbols$pf_Fidelity <- 0
# Fixed income
df.symbols[df.symbols$string.symbol=='FXAIX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='HNACX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='VEIRX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='FSMAX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='PRVIX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='FTIHX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='MDIZX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='DODIX',]$pf_Fidelity <-  1.0/26.0
df.symbols[df.symbols$string.symbol=='FXNAX',]$pf_Fidelity <-  1.0/26.0

# Inflation hedges
df.symbols[df.symbols$string.symbol=='BFIGX',]$pf_LPL <-  4/26
df.symbols[df.symbols$string.symbol=='GOLD',]$pf_LPL <-  4/26

# Equity/Equity Linked
df.symbols[df.symbols$string.symbol=='GSFTX',]$pf_LPL <-  3/26
df.symbols[df.symbols$string.symbol=='PRWCX',]$pf_LPL <-  3/26
df.symbols[df.symbols$string.symbol=='ADOZX',]$pf_LPL <-  3/26


This part normalizes all the data, in case there is a typo or numerical rounding that results in somthing less than 100%


In [ ]:
for (col_name in names(df.symbols)){
  
  if( length(grep('pf_', col_name))){
    df.symbols[,col_name] <- df.symbols[,col_name] / sum(df.symbols[,col_name])
  }
  
}


Update the returns

In [ ]:
df.data <- pfUpdateReturn("pf_Roth_Legacy", df.data, df.symbols)
df.data <- pfUpdateReturn("pf_Roth_IRA", df.data, df.symbols)
df.data <- pfUpdateReturn("pf_LPL", df.data, df.symbols)

# Display the portfolios


Display the two portfolios. A great resource for kable formatting: https://haozhu233.github.io/kableExtra/awesome_table_in_html.html

In [ ]:
string.colnames <- colnames(df.symbols);
string.colnames[1] <- "Symbol"
string.colnames[2] <- "Source"
string.colnames[3] <- "Description"
string.colnames[4] <- "Label"
string.colnames[5] <- "Expense Ratio (%)"
string.colnames[6] <- "Series Start"
string.colnames[7] <- "Expected Return"

## Legacy portfolio

### Portfolio description

In [ ]:
kable(df.symbols[df.symbols$pf_Roth_Legacy > 0.0, ],
      col.names = string.colnames,
      digits = c(0, 0, 0, 0, 2, 0, 2, 2, 3, 3, 3, 4)) %>%
  kable_styling(bootstrap_options = c("striped", "hover")) %>%
  as.character() %>%
  display_html()

### Time series analysis

This one is performing much worse than I expected. Data in this section is for reference only.

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
datay.legacy <- "NOBL.Close_Norm"
ylim <- c(0, 4)
plotSingleBench(dfRecession, datay.legacy, ylim, df.symbols, df.data, string.analysis.start)

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
datay.legacy <- "pf_Roth_Legacy"
ylim <- c(0, 5)
plotSingleBench(dfRecession, datay.legacy, ylim, df.symbols, df.data, string.analysis.start)

### Risk return analysis

This section plots the funds on return and volatility axis. An ideal fund would be in the upper left corner of the plot.

In [ ]:
rrData.legacy <- plotReturnVolatility(datay.legacy, df.data, df.symbols)

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
rrData.legacy[[2]]

This table shows the portfolio. Column `r datay.legacy` shows the inputted portfolio and `r paste(datay.legacy,"_Opt", sep="")` shows the portfolio with the same returns, but less risk.

In [ ]:
tblPort(rrData.legacy, datay.legacy)  %>%
  as.character() %>%
  display_html()

In [ ]:
 df.port <- rrData.legacy[[1]][c("string.symbol", "string.description", "date.series.start", "float.expense.ratio",
                      datay.legacy, paste(datay.legacy, "_Opt", sep = ""))]

In [ ]:
rrData.legacy[[1]]

Setup the optimized portfolio and display the time series

In [ ]:
datay.legacy.opt <- "pf_Roth_Legacy_Opt"
df.data <- pfUpdateReturn(datay.legacy.opt, df.data, df.symbols)

In [ ]:
tail(df.data)

In [ ]:
options(repr.plot.width = 5.7, repr.plot.height = 4, repr.plot.res = 300)
ylim <- c(0, 5)
plotSingleBench(dfRecession, datay.legacy.opt, ylim, df.symbols, df.data, string.analysis.start)

## IRA Roth Portfolio
### Portfolio description

In [ ]:
kable(df.symbols[df.symbols$pf_Roth_IRA > 0.0, ],
      col.names = string.colnames,
      digits = c(0, 0, 0, 0, 2, 0, 2, 2, 3, 3)) %>%
  kable_styling(bootstrap_options = c("striped", "hover")) %>%
  as.character() %>%
  display_html()

### Time series analysis

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
datay.legacy <- "pf_Roth_IRA"
ylim <- c(0, 4)
plotSingleBench(dfRecession, datay.legacy, ylim, df.symbols, df.data, string.analysis.start)

### Risk return analysis

This section plots the funds on return and volatility axis. An ideal fund would be in the upper left corner of the plot.

In [ ]:
rrData.legacy <- plotReturnVolatility(datay.legacy, df.data, df.symbols)

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
rrData.legacy[[2]]

In [ ]:
tblPort(rrData.legacy, datay.legacy)  %>%
  as.character() %>%
  display_html()

In [ ]:
 df.port <- rrData.legacy[[1]][c("string.symbol", "string.description", "date.series.start", "float.expense.ratio",
                      datay.legacy, paste(datay.legacy, "_Opt", sep = ""))]

In [ ]:
rrData.legacy[[1]]

Setup the optimized portfolio and display the time series

In [ ]:
datay.legacy.opt <- "pf_Roth_IRA_Opt"
df.data <- pfUpdateReturn(datay.legacy.opt, df.data, df.symbols)

In [ ]:
tail(df.data)

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
ylim <- c(0, 4)
plotSingleBench(dfRecession, datay.legacy, ylim, df.symbols, df.data, string.analysis.start)

### Stop loss bounds

In [ ]:
# returns for the portfolio
for (str.sym in rrData.legacy[[1]]$string.symbol){
    
    # Create the actual field names
    str.sym.lookup <- paste(str.sym, '.Close', sep="");
    print(str.sym)
    str.sym.store <- paste(str.sym, '.Close_ClCl', sep="")
    print(str.sym.store)
    str.desc <- df.symbols[grep(paste('^',str.sym,'$', sep=""), df.symbols$string.symbol),]$string.description
    print(str.desc)
    
    df.data[str.sym.store] <- quantmod::ClCl(df.data[str.sym.lookup])*100
    df.data[str.sym.store][is.na(df.data[str.sym.store])] <- 0
    df.symbols <-
      rbind(
        df.symbols,
        data.frame(
          string.symbol = str.sym.store,
          string.source = "Calc",
          string.description =  paste(str.desc, '(ClCl)'),
          string.label.y = "Percent",
          float.expense.ratio = -1.00,
          date.series.start = df.symbols[grep(paste('^',str.sym,'$', sep=""), df.symbols$string.symbol),]$date.series.start,
          date.series.end = df.symbols[grep(paste('^',str.sym,'$', sep=""), df.symbols$string.symbol),]$date.series.end,
          Max030 = FALSE,
          Max180 = FALSE,
          ExpReturn = 0,
          Volatility = 0,
          pf_Roth_Legacy = 0,
          pf_Roth_IRA = 0,
          pf_LPL = 0
        )
      )
    
}

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 9, repr.plot.res = 600)
datay <- "NOBL.Close_ClCl"
ylim <- c(-5, 10)
b.legend <- FALSE
b.percentile <- FALSE
p1 <-
  plotSingle(
    dfRecession,
    df.data,
    "date",
    datay,
    getPlotTitle(df.symbols, datay),
    "Date",
    getPlotYLabel(df.symbols, datay),
    c(dt.recent, Sys.Date()),
    ylim,
    b.legend,
    b.percentile = TRUE
  )

datay <- "SCHD.Close_ClCl"
ylim <- c(-5, 10)
b.legend <- FALSE
b.percentile <- TRUE
p2 <-
  plotSingle(
    dfRecession,
    df.data,
    "date",
    datay,
    getPlotTitle(df.symbols, datay),
    "Date",
    getPlotYLabel(df.symbols, datay),
    c(dt.recent, Sys.Date()),
    ylim,
    b.legend,
    b.percentile
  )

datay <- "GDX.Close_ClCl"
ylim <- c(-5, 10)
b.legend <- TRUE
b.percentile <- TRUE
p3 <-
  plotSingle(
    dfRecession,
    df.data,
    "date",
    datay,
    getPlotTitle(df.symbols, datay),
    "Date",
    getPlotYLabel(df.symbols, datay),
    c(dt.recent, Sys.Date()),
    ylim,
    b.legend,
    b.percentile
  )

grid.arrange(p1, p2, p3, ncol = 1, top = "Historical performance of IRA/ROTH portfolio")

## Amy's Fidelity Portfolio

In [ ]:
kable(df.symbols[df.symbols$pf_Fidelity > 0.0, ],
      col.names = string.colnames,
      digits = c(0, 0, 0, 0, 2, 0, 2, 2, 3, 3)) %>%
  kable_styling(bootstrap_options = c("striped", "hover")) %>%
  as.character() %>%
  display_html()

### Time series analysis

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
datay.legacy <- "pf_Fidelity"
ylim <- c(0, 5)
plotSingleBench(dfRecession, datay.legacy, ylim, df.symbols, df.data, string.analysis.start)

In [ ]:
rrData <- plotReturnVolatility(datay.legacy, df.data, df.symbols)

In [ ]:
options(repr.plot.width = 9.7, repr.plot.height = 6, repr.plot.res = 600)
rrData[[2]]

This table shows the portfolio. Column `r datay.legacy` shows the inputted portfolio and `r paste(datay.legacy,"_Opt", sep="")` shows the portfolio with the same returns, but less risk.

In [ ]:
tblPort(rrData, datay.legacy)  %>%
  as.character() %>%
  display_html()